# CRF-CORA

This file is the first attempt at using pystruct.GraphCRF to do CRF for the clean cora dataset

In [1]:
import pickle
import gensim

from pystruct.models import GraphCRF
from pystruct.learners import OneSlackSSVM
from gensim.models import Word2Vec

In [2]:
data_path = './data/'

In [3]:
with open(data_path+'cleaned_dataset.pkl', 'rb') as f:
    data = pickle.load(f)

for id in data['features_for_id']:
    sentence = ""
    for feature in data['features_for_id'][id]:
        word = feature.split('=')[1]
        if word == 'missing':
            continue
        else:
            sentence += feature.split('=')[1] + ' '
    data['features_for_id'][id] = sentence.strip()

In [4]:
features_list = [data['features_for_id'][id].split() for id in data['features_for_id']]

In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

for i in range(len(features_list)):
    features_list[i] = [word.lower() for word in features_list[i] if word.isalpha()]
    features_list[i] = [word for word in features_list[i] if not word in stop_words]

In [6]:
model = Word2Vec(features_list, min_count=1)

In [7]:
print(model)

Word2Vec(vocab=21669, size=100, alpha=0.025)


In [9]:
words = list(model.wv.vocab)
print(words)

['thomas', 'ellman', 'synthesis', 'abstraction', 'hierarchies', 'constraint', 'satisfaction', 'clustering', 'approximately', 'equivalent', 'objects', 'otoole', 'shrira', 'hybrid', 'caching', 'large', 'scale', 'object', 'systems', 'lindsay', 'empirical', 'studies', 'genetic', 'algorithm', 'noncoding', 'segments', 'leonidas', 'kontothanassis', 'robert', 'wisniewski', 'michael', 'scott', 'schedulerconscious', 'synchronization', 'steve', 'ward', 'numesh', 'scalable', 'modular', 'interconnect', 'david', 'mosberger', 'larry', 'peterson', 'patrick', 'bridges', 'sean', 'omalley', 'analysis', 'techniques', 'improve', 'protocol', 'latency', 'tadepalli', 'autoexploratory', 'average', 'reward', 'reinforcement', 'learning', 'stewart', 'perturbation', 'pseudoinverses', 'projections', 'linear', 'least', 'squares', 'problems', 'siam', 'review', 'gokhale', 'schmidt', 'evaluating', 'performance', 'demultiplexing', 'strategies', 'realtime', 'corba', 'ieee', 'phoenixaz', 'shokrollahi', 'wasserman', 'decod